In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam

import numpy as np
import os

LEARNING_RATE = 0.0005
BATCH_SIZE = 128
EPOCHS = 12

In [4]:
import sys
sys.path.insert(1, 'D:\Code\Data Science\Projects\music_generador_deep_learning\src/modeling')

from vq_vae import VQ_VAE
from train_vq import *

In [9]:
SPECTROGRAMS_PATH = "D:\Code\Data Science\Projects\music_generador_deep_learning\data/fsdd/spectrograms/"

LEARNING_RATE = 0.0005
BATCH_SIZE = 64
EPOCHS = 150

In [10]:
x_train, _ = load_fsdd(SPECTROGRAMS_PATH)
x_train

In [13]:
data_variance = np.var(x_train / 255.0)

In [14]:
VQVAE = VQ_VAE(
    input_shape=(256, 64, 1),
    conv_filters=(512, 256, 128, 64, 32),
    conv_kernels=(3, 3, 3, 3, 3),
    conv_strides=(2, 2, 2, 2, (2, 1)),
    data_variance=data_variance,
    latent_space_dim=128
)
VQVAE.summary()


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)      │ (None, 256, 64, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_layer_0 (Conv2D)           │ (None, 128, 32, 512)   │         5,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_relu_0 (ReLU)           │ (None, 128, 32, 512)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_bn_0                    │ (None, 128, 32, 512)   │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_layer_1 (Conv2D)           │ (None, 64, 16, 256)    │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_relu_1 (ReLU)           │ (None, 64, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_bn_1                    │ (None, 64, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_layer_2 (Conv2D)           │ (None, 32, 8, 128)     │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_relu_2 (ReLU)           │ (None, 32, 8, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_bn_2                    │ (None, 32, 8, 128)     │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_layer_3 (Conv2D)           │ (None, 16, 4, 64)      │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_relu_3 (ReLU)           │ (None, 16, 4, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_bn_3                    │ (None, 16, 4, 64)      │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_layer_4 (Conv2D)           │ (None, 8, 4, 32)       │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_relu_4 (ReLU)           │ (None, 8, 4, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_bn_4                    │ (None, 8, 4, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 8, 4, 128)      │         4,224 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,580,512 (6.03 MB)

 Trainable params: 1,578,528 (6.02 MB)

 Non-trainable params: 1,984 (7.75 KB)

...

Model: "variational_autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)      │ (None, 256, 64, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Functional)            │ (None, 8, 4, 128)      │     1,580,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vector_quantizer                │ (None, 8, 4, 128)      │             0 │
│ (VectorQuantizer)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ (None, 256, 64, 1)     │       428,641 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,009,153 (7.66 MB)

 Trainable params: 2,006,209 (7.65 MB)

 Non-trainable params: 2,944 (11.50 KB)

In [15]:
VQVAE.compile(LEARNING_RATE)

In [16]:
VQVAE.train(x_train, BATCH_SIZE, EPOCHS)

Epoch 1/150
47/47 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - reconstruction_loss: 240168.9219 - total_loss: 242978.5312 - vq_loss: 2800.3130
Epoch 2/150
47/47 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - reconstruction_loss: 61598.9180 - total_loss: 54275272.0000 - vq_loss: 54213668.0000
Epoch 3/150
47/47 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - reconstruction_loss: 35106.4062 - total_loss: 907982602240.0000 - vq_loss: 907982536704.0000
Epoch 4/150
47/47 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - reconstruction_loss: 32385.7637 - total_loss: 9163693359104.0000 - vq_loss: 9163693359104.0000
Epoch 5/150
47/47 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - reconstruction_loss: 26769.8867 - total_loss: 11142450642944.0000 - vq_loss: 11142450642944.0000
Epoch 6/150
47/47 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - reconstruction_loss: 39205.3750 - total_loss: 1318574287224832.0000 - vq_loss: 1318574287224832.0000
Epoch 7/150
47/47 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - reconstruction_loss: 74236.0469 - total_loss: 2560194840100864.0000 - vq_loss:

In [17]:
VQVAE.save("model/vq_vae")

NameError: name 'VQVAE' is not defined